# Using the Waffle House API to scrape restaurant closure data

### Setup

In [5]:
import requests
import pandas as pd

In [6]:
# United States of America Python Dictionary to translate States,
# Districts & Territories to Two-Letter codes and vice versa.
#
# https://gist.github.com/rogerallen/1583593
#
# Dedicated to the public domain.  To the extent possible under law,
# Roger Allen has waived all copyright and related or neighboring
# rights to this code.

us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

# thank you to @kinghelix and @trevormarburger for this idea
abbrev_us_state = dict(map(reversed, us_state_abbrev.items()))

# Simple test examples
if __name__ == '__main__':
    print("Wisconin --> WI?", us_state_abbrev['Wisconsin'] == 'WI')
    print("WI --> Wisconin?", abbrev_us_state['WI'] == 'Wisconsin')
    print("Number of entries (50 states, DC, 5 Territories) == 56? ", 56 == len(us_state_abbrev))

Wisconin --> WI? True
WI --> Wisconin? True
Number of entries (50 states, DC, 5 Territories) == 56?  True


In [7]:
branch=[] #List to store name of the Waffle House location
lat=[] #latitude, second coordinate in the data
lon=[] #longitude, first coordinate in the data
streetAddress=[] #List to store address street name of the Waffle House location
city=[] #List to store address city of the Waffle House location
state=[] #List to store address region/state of the Waffle House location
zipCode=[] #List to store postal code of the Waffle House location
country=[] #List to store country of the Waffle House location
isClosed=[] #List to store closure data of the Waffle House location

### Main Code

In [8]:
#Scrape all the required data
url = "https://locations.wafflehouse.com/api/587d236eeb89fb17504336db/locations-details?locale=en_US"
locations = requests.get(url).json()
for loc in range(0,len(locations["features"])):
    branch.append(locations["features"][loc]['properties']['branch'])
    lat.append(locations["features"][loc]['geometry']['coordinates'][1])
    lon.append(locations["features"][loc]['geometry']['coordinates'][0])
    streetAddress.append(locations["features"][loc]['properties']['addressLine1'])
    city.append(locations["features"][loc]['properties']['city'])
    state.append(locations["features"][loc]['properties']['province'])
    zipCode.append(locations["features"][loc]['properties']['postalCode'])
    country.append(locations["features"][loc]['properties']['country'])        
    if(len(locations["features"][loc]['properties']['specialHoursOfOperation']) > 0):
        isClosed.append(locations["features"][loc]['properties']['specialHoursOfOperation'][0]['isClosed'])
    else: 
        isClosed.append(False)
        #to improve this and flag reduced hours stores, need to add "24Hours" column indictor variable[0]['_24hrs']

In [9]:
#fix the mixed state name/state abbreviations in "state" variable
for s in range(0,len(state)):
    if len(state[s]) ==2:
        state[s] = abbrev_us_state[state[s]].upper()

In [10]:
#Store the data
df = pd.DataFrame({'Branch':branch,'Latitude':lat,'Longitude':lon,'Address':streetAddress,'City':city, \
                   'State':state,'Zip Code':zipCode,'Country':country,'isClosed':isClosed}) 
df.to_csv('Documents/Code/Datasets/Wafflehouse/wafflehouse.csv', index=False, encoding='utf-8')

### Testing and Practice

In [42]:
locations

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [-81.524179, 35.91727]},
   'properties': {'name': 'Waffle House',
    'id': '57129dbbc49072572ed4ab15',
    'branch': '1976',
    'slug': 'nc-lenoir-1976',
    'addressLine1': '220 Blowing Rock Blvd',
    'addressLine2': '',
    'city': 'Lenoir',
    'province': 'NORTH CAROLINA',
    'postalCode': '28645',
    'country': 'UNITED STATES',
    'phoneNumber': '+1 828-754-5810',
    'phoneLabel': '(828) 754-5810',
    'hoursOfOperation': {'Mon': [['00:00', '23:59']],
     'Tue': [['00:00', '23:59']],
     'Wed': [['00:00', '23:59']],
     'Thu': [['00:00', '23:59']],
     'Fri': [['00:00', '23:59']],
     'Sat': [['00:00', '23:59']],
     'Sun': [['00:00', '23:59']]},
    'cmsFeatures': [{'fieldGroup': 'Address',
      'component': 'Address',
      'props': {'addressLine1': '{{ dataLocations.selectedLocation.properties.addressLine1 | connect }}',
       'addressLine2': '{{ dataL

In [103]:
locations["features"][0]

{'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [-81.524179, 35.91727]},
 'properties': {'name': 'Waffle House',
  'id': '57129dbbc49072572ed4ab15',
  'branch': '1976',
  'slug': 'nc-lenoir-1976',
  'addressLine1': '220 Blowing Rock Blvd',
  'addressLine2': '',
  'city': 'Lenoir',
  'province': 'NORTH CAROLINA',
  'postalCode': '28645',
  'country': 'UNITED STATES',
  'phoneNumber': '+1 828-754-5810',
  'phoneLabel': '(828) 754-5810',
  'hoursOfOperation': {'Mon': [['00:00', '23:59']],
   'Tue': [['00:00', '23:59']],
   'Wed': [['00:00', '23:59']],
   'Thu': [['00:00', '23:59']],
   'Fri': [['00:00', '23:59']],
   'Sat': [['00:00', '23:59']],
   'Sun': [['00:00', '23:59']]},
  'cmsFeatures': [{'fieldGroup': 'Address',
    'component': 'Address',
    'props': {'addressLine1': '{{ dataLocations.selectedLocation.properties.addressLine1 | connect }}',
     'addressLine2': '{{ dataLocations.selectedLocation.properties.addressLine2 | connect }}',
     'city': '{{ dataLocati

In [87]:
locations["features"][0]['properties']

{'name': 'Waffle House',
 'id': '57129dbbc49072572ed4ab15',
 'branch': '1976',
 'slug': 'nc-lenoir-1976',
 'addressLine1': '220 Blowing Rock Blvd',
 'addressLine2': '',
 'city': 'Lenoir',
 'province': 'NORTH CAROLINA',
 'postalCode': '28645',
 'country': 'UNITED STATES',
 'phoneNumber': '+1 828-754-5810',
 'phoneLabel': '(828) 754-5810',
 'hoursOfOperation': {'Mon': [['00:00', '23:59']],
  'Tue': [['00:00', '23:59']],
  'Wed': [['00:00', '23:59']],
  'Thu': [['00:00', '23:59']],
  'Fri': [['00:00', '23:59']],
  'Sat': [['00:00', '23:59']],
  'Sun': [['00:00', '23:59']]},
 'cmsFeatures': [{'fieldGroup': 'Address',
   'component': 'Address',
   'props': {'addressLine1': '{{ dataLocations.selectedLocation.properties.addressLine1 | connect }}',
    'addressLine2': '{{ dataLocations.selectedLocation.properties.addressLine2 | connect }}',
    'city': '{{ dataLocations.selectedLocation.properties.city | connect }}',
    'province': '{{ dataLocations.selectedLocation.properties.province | conn